# Competition Recommender Systems

In [1]:
from google.colab import drive
drive.mount('/content/drive', True)

Mounted at /content/drive


In [2]:
import sys
sys.path.insert(0,'/content/drive/MyDrive/Competition_Recommender_Systems')

In [3]:
import os
from typing import Tuple, Callable, Dict, Optional, List

import numpy as np
import pandas as pd
import scipy.sparse as sp

from sklearn.model_selection import train_test_split
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

#np.random.seed(1234)

cwd = '/content/drive/MyDrive/Competition_Recommender_Systems'

In [4]:
def load_data(file_path, load_users=False):
    data = pd.read_csv(file_path)
    
    if load_users:
        user_list = data['user_id'].tolist()
        return user_list
    else:
        user_list = data['row'].tolist()
        item_list = data['col'].tolist()
        rating_list = data['data'].tolist()
        return sp.coo_matrix((rating_list, (user_list, item_list))).tocsr()

ratings = load_data(os.path.join(cwd, 'dataset', 'data_train.csv'))

icm_all = load_data(os.path.join(cwd, 'dataset', 'data_ICM_title_abstract.csv'))

target_users = load_data(os.path.join(cwd, 'dataset', 'data_target_users_test.csv'), load_users=True)

for i in range(7947):
    if i not in target_users:
        print("Missing user: {}".format(i))

Missing user: 1304
Missing user: 2245
Missing user: 4319


In [5]:
urm_train, urm_validation = split_train_in_two_percentage_global_sample(ratings, 0.8)

In [6]:
urm_train

<7947x25975 sparse matrix of type '<class 'numpy.float64'>'
	with 90614 stored elements in Compressed Sparse Row format>

In [7]:
urm_validation

<7947x25975 sparse matrix of type '<class 'numpy.float64'>'
	with 22654 stored elements in Compressed Sparse Row format>

In [8]:
URM_train_t = urm_train.T
ICM_and_URM = sp.hstack([icm_all, URM_train_t], format="csr")
URM_and_ICM = sp.vstack([urm_train, icm_all.T], format="csr")

## Import the evaluator object and define the parameters

In [ ]:
from Base.Evaluation.Evaluator import EvaluatorHoldout

evaluator_validation = EvaluatorHoldout(urm_validation, cutoff_list=[5])

In [ ]:
# Rec.1 = ItemKNNCF
best_parameters_rec_1 = {'topK': 462,
                         'shrink': 3178,
                         'similarity': 'cosine',
                         'normalize': True,
                         'feature_weighting': 'TF-IDF'}

# Rec.2 = UserKNNCF
best_parameters_rec_2 = {'topK': 201,
                         'shrink': 419, 
                         'similarity': 'cosine',
                         'normalize': True,
                         'feature_weighting': 'TF-IDF'}

# Rec.3 = ItemKNNCBF
best_parameters_rec_3 = {'topK': 2993,
                         'shrink': 246,
                         'similarity': 'cosine',
                         'normalize': True,
                         'feature_weighting': 'TF-IDF'}

# Rec.4 = PureSVDItem
best_parameters_rec_4 = {'topK': 1500, 
                         'num_factors': 1216}

# Rec.5 = P3alpha
best_parameters_rec_5 = {'topK': 298,
                         'alpha': 0.60115435676804,
                         'implicit': True,
                         'normalize_similarity': False}

# Rec.6 = SLIM_BPR
best_parameters_rec_6 = {"batch_size": 548, 
                         "symmetric": False, 
                         "sgd_mode": 'adagrad',
                         "epochs": 98, 
                         "lambda_i": 0.0039954585909894764,
                         "lambda_j": 0.008139187451093313,
                         "topK": 591}

# Rec.7 = RP3beta
best_parameters_rec_7 = {'alpha': 0.38770822445994335,
                         'beta': 0.3570967475164035,
                         'topK': 2959,
                         'implicit': True,
                         'normalize_similarity': False}

# Rec.8 = CFW
best_parameters_rec_8 = {'topK': 239,
                         'add_zeros_quota': 0.11545813905415524,
                         'normalize_similarity': False,
                         'damp_coeff': 0.49149802139393484,
                         'loss_tolerance': 1e-06}

# Rec. 9 = iALS 
best_parameters_rec_9 = {'alpha': 28,
                         'confidence_scaling': 'linear',
                         'iterations': 200,
                         'n_factors': 622,
                         'reg': 0.030756138926585266,
                         'use_gpu': True}

# Rec. 10 = FunkSVD 
best_parameters_rec_10 = {'batch_size': 105,
                          'num_factors': 14,
                          'learning_rate': 0.00010115830807576066,
                          'sgd_mode': 'adam',
                          'epochs': 10}

# Rec. 11 = AsymmetricSVD 
best_parameters_rec_11 = {'batch_size': 1,
                          'num_factors': 10,
                          'learning_rate': 0.1,
                          'sgd_mode': 'adam',
                          'epochs': 1}

# Rec. 12 = MF_BPR 
best_parameters_rec_12 = {'batch_size': 2048,
                          'num_factors': 200,
                          'learning_rate': 0.1,
                          'sgd_mode': 'adagrad',
                          'epochs': 164}

# Rec. 13 = SLIM_ElasticNet
best_parameters_rec_13 = {'alpha': 0.004351486558816739, 
                          'l1_ratio': 2.0173100409475224e-05, 
                          'topK': 999}

# Rec. 14 = ItemKNNCBF with stack
best_parameters_rec_14 = {'topK': 705, 
                          'shrink': 635,
                          'similarity': 'asymmetric', 
                          'feature_weighting': 'TF-IDF'}

### ItemKNNCF

In [ ]:
from KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
itemKNNCF = ItemKNNCFRecommender(URM_and_ICM)
itemKNNCF.fit(**best_parameters_rec_1)
evaluator_validation.evaluateRecommender(itemKNNCF)

ItemKNNCFRecommender: URM Detected 65 (0.23 %) cold users.
ItemKNNCFRecommender: URM Detected 2 (0.01 %) cold items.
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 25975 ( 100 % ), 970.31 column/sec, elapsed time 0.45 min
EvaluatorHoldout: Processed 5596 ( 100.00% ) in 3.82 sec. Users per second: 1466


({5: {'ARHR': 0.14726590421729896,
   'AVERAGE_POPULARITY': 0.153125495044723,
   'COVERAGE_ITEM': 0.2864677574590953,
   'COVERAGE_ITEM_CORRECT': 0.03337824831568816,
   'COVERAGE_USER': 0.7041650937460677,
   'COVERAGE_USER_CORRECT': 0.15615955706555934,
   'DIVERSITY_GINI': 0.10083991315016028,
   'DIVERSITY_HERFINDAHL': 0.9979809866232441,
   'DIVERSITY_MEAN_INTER_LIST': 0.9900818598245523,
   'F1': 0.06939346983123529,
   'HIT_RATE': 0.27090779127948533,
   'MAP': 0.06506065840679827,
   'MRR': 0.13252620919704608,
   'NDCG': 0.0831046459442131,
   'NOVELTY': 0.0026382087031505256,
   'PRECISION': 0.054181558255896003,
   'PRECISION_RECALL_MIN_DEN': 0.10799976173457215,
   'RECALL': 0.0964814066460539,
   'ROC_AUC': 0.1369579461520134,
   'SHANNON_ENTROPY': 11.002620729686935}},
 'CUTOFF: 5 - ROC_AUC: 0.1369579, PRECISION: 0.0541816, PRECISION_RECALL_MIN_DEN: 0.1079998, RECALL: 0.0964814, MAP: 0.0650607, MRR: 0.1325262, NDCG: 0.0831046, F1: 0.0693935, HIT_RATE: 0.2709078, ARHR: 0.

### UserKNNCF

In [ ]:
from KNN.UserKNNCFRecommender import UserKNNCFRecommender
userKNNCF = UserKNNCFRecommender(URM_and_ICM)
userKNNCF.fit(**best_parameters_rec_2)
evaluator_validation.evaluateRecommender(userKNNCF)

UserKNNCFRecommender: URM Detected 83 (0.30 %) cold users.
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 27947 ( 100 % ), 1122.71 column/sec, elapsed time 0.41 min
EvaluatorHoldout: Processed 5664 ( 100.00% ) in 3.33 sec. Users per second: 1703


({5: {'ARHR': 0.15270127118644167,
   'AVERAGE_POPULARITY': 0.1607953240549361,
   'COVERAGE_ITEM': 0.28662175168431187,
   'COVERAGE_ITEM_CORRECT': 0.034340712223291624,
   'COVERAGE_USER': 0.7127217818044546,
   'COVERAGE_USER_CORRECT': 0.16157040392600983,
   'DIVERSITY_GINI': 0.10047982416189477,
   'DIVERSITY_HERFINDAHL': 0.9979571169084555,
   'DIVERSITY_MEAN_INTER_LIST': 0.9899603656802856,
   'F1': 0.07195117975682577,
   'HIT_RATE': 0.2791313559322034,
   'MAP': 0.06964762829566844,
   'MRR': 0.13716454802259948,
   'NDCG': 0.08776069917659522,
   'NOVELTY': 0.0026299738065367925,
   'PRECISION': 0.05582627118643957,
   'PRECISION_RECALL_MIN_DEN': 0.11290901600753281,
   'RECALL': 0.10117451413782519,
   'ROC_AUC': 0.141551906779661,
   'SHANNON_ENTROPY': 10.986918685730576}},
 'CUTOFF: 5 - ROC_AUC: 0.1415519, PRECISION: 0.0558263, PRECISION_RECALL_MIN_DEN: 0.1129090, RECALL: 0.1011745, MAP: 0.0696476, MRR: 0.1371645, NDCG: 0.0877607, F1: 0.0719512, HIT_RATE: 0.2791314, ARHR: 

### ItemKNNCBF

In [ ]:
from KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
itemKNNCBF = ItemKNNCBFRecommender(URM_and_ICM, icm_all)
itemKNNCBF.fit(**best_parameters_rec_3)
evaluator_validation.evaluateRecommender(itemKNNCBF)

ItemKNNCBFRecommender: URM Detected 65 (0.23 %) cold users.
ItemKNNCBFRecommender: URM Detected 2 (0.01 %) cold items.
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 21500 ( 83 % ), 716.55 column/sec, elapsed time 0.50 min
Similarity column 25975 ( 100 % ), 716.30 column/sec, elapsed time 0.60 min
EvaluatorHoldout: Processed 5596 ( 100.00% ) in 5.34 sec. Users per second: 1049


({5: {'ARHR': 0.06686621396235386,
   'AVERAGE_POPULARITY': 0.07718669706160754,
   'COVERAGE_ITEM': 0.4268719923002887,
   'COVERAGE_ITEM_CORRECT': 0.025948026948989414,
   'COVERAGE_USER': 0.7041650937460677,
   'COVERAGE_USER_CORRECT': 0.07914936454007802,
   'DIVERSITY_GINI': 0.2291648771076529,
   'DIVERSITY_HERFINDAHL': 0.9996705320506172,
   'DIVERSITY_MEAN_INTER_LIST': 0.9985310968322196,
   'F1': 0.03349214328067479,
   'HIT_RATE': 0.13009292351679771,
   'MAP': 0.029045697323484967,
   'MRR': 0.06138610912556575,
   'NDCG': 0.038653450334022586,
   'NOVELTY': 0.0027892510546512236,
   'PRECISION': 0.026018584703359604,
   'PRECISION_RECALL_MIN_DEN': 0.0522337383845602,
   'RECALL': 0.04698931998922792,
   'ROC_AUC': 0.06403383369073146,
   'SHANNON_ENTROPY': 12.712645595584295}},
 'CUTOFF: 5 - ROC_AUC: 0.0640338, PRECISION: 0.0260186, PRECISION_RECALL_MIN_DEN: 0.0522337, RECALL: 0.0469893, MAP: 0.0290457, MRR: 0.0613861, NDCG: 0.0386535, F1: 0.0334921, HIT_RATE: 0.1300929, AR

### PureSVDItem

In [ ]:
from MatrixFactorization.PureSVDRecommender import PureSVDItemRecommender
pureSVD = PureSVDItemRecommender(urm_train)
pureSVD.fit(**best_parameters_rec_4)
evaluator_validation.evaluateRecommender(pureSVD)

PureSVDItemRecommender: URM Detected 63 (0.79 %) cold users.
PureSVDItemRecommender: URM Detected 2346 (9.03 %) cold items.
PureSVDItemRecommender: Computing SVD decomposition...
PureSVDItemRecommender: Computing SVD decomposition... Done!
EvaluatorHoldout: Processed 5596 ( 100.00% ) in 7.16 sec. Users per second: 781


({5: {'ROC_AUC': 0.09036216345008334,
   'PRECISION': 0.03359542530378815,
   'PRECISION_RECALL_MIN_DEN': 0.06342625685013063,
   'RECALL': 0.05472541102822206,
   'MAP': 0.04028780478119285,
   'MRR': 0.08794376935906574,
   'NDCG': 0.049833116832064775,
   'F1': 0.041632836254096944,
   'HIT_RATE': 0.1679771265189421,
   'ARHR': 0.09643197522039523,
   'NOVELTY': 0.002560423330535467,
   'AVERAGE_POPULARITY': 0.03914798997475585,
   'DIVERSITY_MEAN_INTER_LIST': 0.9979042224083206,
   'DIVERSITY_HERFINDAHL': 0.9995451795702127,
   'COVERAGE_ITEM': 0.26086621751684314,
   'COVERAGE_ITEM_CORRECT': 0.0280269489894129,
   'COVERAGE_USER': 0.7041650937460677,
   'COVERAGE_USER_CORRECT': 0.09915691455895306,
   'DIVERSITY_GINI': 0.11981711605786438,
   'SHANNON_ENTROPY': 11.927756080664231}},
 'CUTOFF: 5 - ROC_AUC: 0.0903622, PRECISION: 0.0335954, PRECISION_RECALL_MIN_DEN: 0.0634263, RECALL: 0.0547254, MAP: 0.0402878, MRR: 0.0879438, NDCG: 0.0498331, F1: 0.0416328, HIT_RATE: 0.1679771, ARHR

### P3alpha search

In [ ]:
from GraphBased.P3alphaRecommender import P3alphaRecommender
P3alpha = P3alphaRecommender(urm_train)
P3alpha.fit(**best_parameters_rec_5)
evaluator_validation.evaluateRecommender(P3alpha)

P3alphaRecommender: URM Detected 63 (0.79 %) cold users.
P3alphaRecommender: URM Detected 2346 (9.03 %) cold items.
EvaluatorHoldout: Processed 5596 ( 100.00% ) in 4.63 sec. Users per second: 1208


({5: {'ROC_AUC': 0.12209614010007153,
   'PRECISION': 0.04917798427448096,
   'PRECISION_RECALL_MIN_DEN': 0.09512449368596565,
   'RECALL': 0.08412621508299092,
   'MAP': 0.056633905170359725,
   'MRR': 0.11795627829401979,
   'NDCG': 0.07235678391906568,
   'F1': 0.062070928033236585,
   'HIT_RATE': 0.24588992137240887,
   'ARHR': 0.1314748629973796,
   'NOVELTY': 0.002428735213989101,
   'AVERAGE_POPULARITY': 0.10847655649152642,
   'DIVERSITY_MEAN_INTER_LIST': 0.9912374152097663,
   'DIVERSITY_HERFINDAHL': 0.9982120564009522,
   'COVERAGE_ITEM': 0.2965543792107796,
   'COVERAGE_ITEM_CORRECT': 0.02906641000962464,
   'COVERAGE_USER': 0.7041650937460677,
   'COVERAGE_USER_CORRECT': 0.14156285390713477,
   'DIVERSITY_GINI': 0.11076413968092394,
   'SHANNON_ENTROPY': 11.196020055599881}},
 'CUTOFF: 5 - ROC_AUC: 0.1220961, PRECISION: 0.0491780, PRECISION_RECALL_MIN_DEN: 0.0951245, RECALL: 0.0841262, MAP: 0.0566339, MRR: 0.1179563, NDCG: 0.0723568, F1: 0.0620709, HIT_RATE: 0.2458899, ARHR

### SLIM_BPR search

In [ ]:
from SLIM_BPR.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
slim_BPR = SLIM_BPR_Cython(urm_train)
slim_BPR.fit(**best_parameters_rec_6)
evaluator_validation.evaluateRecommender(slim_BPR)

SLIM_BPR_Recommender: URM Detected 63 (0.79 %) cold users.
SLIM_BPR_Recommender: URM Detected 2346 (9.03 %) cold items.
Unable to read memory status: list index out of range
SLIM_BPR_Recommender: Automatic selection of fastest train mode. Unable to get current RAM status, you may be using a non-Linux operating system. Using dense matrix.
Processed 7947 ( 100.00% ) in 0.17 seconds. BPR loss is 2.92E-04. Sample per second: 46564
SLIM_BPR_Recommender: Epoch 1 of 48. Elapsed time 0.03 sec
Processed 7947 ( 100.00% ) in 0.20 seconds. BPR loss is 1.39E-03. Sample per second: 39817
SLIM_BPR_Recommender: Epoch 2 of 48. Elapsed time 0.06 sec
Processed 7947 ( 100.00% ) in 0.23 seconds. BPR loss is 3.71E-03. Sample per second: 34477
SLIM_BPR_Recommender: Epoch 3 of 48. Elapsed time 0.09 sec
Processed 7947 ( 100.00% ) in 0.26 seconds. BPR loss is 5.26E-03. Sample per second: 30517
SLIM_BPR_Recommender: Epoch 4 of 48. Elapsed time 0.12 sec
Processed 7947 ( 100.00% ) in 0.29 seconds. BPR loss is 7.76

{}

EvaluatorHoldout: Processed 5596 ( 100.00% ) in 4.84 sec. Users per second: 1157


({5: {'ROC_AUC': 0.11217834167262328,
   'PRECISION': 0.04270907791279428,
   'PRECISION_RECALL_MIN_DEN': 0.08331546342625622,
   'RECALL': 0.07437322010584904,
   'MAP': 0.05006542371535212,
   'MRR': 0.10681141291398616,
   'NDCG': 0.06474765157110701,
   'F1': 0.05425946886728029,
   'HIT_RATE': 0.21354538956397426,
   'ARHR': 0.11673814629497284,
   'NOVELTY': 0.0023932739204866995,
   'AVERAGE_POPULARITY': 0.1348269785832554,
   'DIVERSITY_MEAN_INTER_LIST': 0.9859347510445671,
   'DIVERSITY_HERFINDAHL': 0.9971517130841442,
   'COVERAGE_ITEM': 0.2618671799807507,
   'COVERAGE_ITEM_CORRECT': 0.022059672762271415,
   'COVERAGE_USER': 0.7041650937460677,
   'COVERAGE_USER_CORRECT': 0.12746948534038002,
   'DIVERSITY_GINI': 0.09038003794342081,
   'SHANNON_ENTROPY': 10.776317523359658}},
 'CUTOFF: 5 - ROC_AUC: 0.1121783, PRECISION: 0.0427091, PRECISION_RECALL_MIN_DEN: 0.0833155, RECALL: 0.0743732, MAP: 0.0500654, MRR: 0.1068114, NDCG: 0.0647477, F1: 0.0542595, HIT_RATE: 0.2135454, ARHR

### RP3beta

In [ ]:
from GraphBased.RP3betaRecommender import RP3betaRecommender
RP3beta = RP3betaRecommender(URM_and_ICM)
RP3beta.fit(**best_parameters_rec_7)
evaluator_validation.evaluateRecommender(RP3beta)

RP3betaRecommender: URM Detected 83 (0.30 %) cold users.
RP3betaRecommender: Processed 24600 ( 94.71% ) in 1.01 minutes. Rows per second: 407
EvaluatorHoldout: Processed 5664 ( 100.00% ) in 4.98 sec. Users per second: 1137


({5: {'ARHR': 0.1544697504708108,
   'AVERAGE_POPULARITY': 0.12356508275534724,
   'COVERAGE_ITEM': 0.36692974013474494,
   'COVERAGE_ITEM_CORRECT': 0.03996150144369586,
   'COVERAGE_USER': 0.7127217818044546,
   'COVERAGE_USER_CORRECT': 0.16282874040518436,
   'DIVERSITY_GINI': 0.1660796406591743,
   'DIVERSITY_HERFINDAHL': 0.9990611434294104,
   'DIVERSITY_MEAN_INTER_LIST': 0.9954814730568433,
   'F1': 0.07281376873240225,
   'HIT_RATE': 0.2791313559322034,
   'MAP': 0.07041004590395473,
   'MRR': 0.13882709510357857,
   'NDCG': 0.08906616249840607,
   'NOVELTY': 0.0027095922804331895,
   'PRECISION': 0.05582627118643957,
   'PRECISION_RECALL_MIN_DEN': 0.11579272598870059,
   'RECALL': 0.10466141997659302,
   'ROC_AUC': 0.1417137476459511,
   'SHANNON_ENTROPY': 11.980927982883586}},
 'CUTOFF: 5 - ROC_AUC: 0.1417137, PRECISION: 0.0558263, PRECISION_RECALL_MIN_DEN: 0.1157927, RECALL: 0.1046614, MAP: 0.0704100, MRR: 0.1388271, NDCG: 0.0890662, F1: 0.0728138, HIT_RATE: 0.2791314, ARHR: 0

### CFW

In [ ]:
from FeatureWeighting.CFW_D_Similarity_Linalg import CFW_D_Similarity_Linalg
CFW = CFW_D_Similarity_Linalg(urm_train, ICM_and_URM, itemKNNCF.W_sparse)
CFW.fit(**best_parameters_rec_8)
evaluator_validation.evaluateRecommender(CFW)

CFW_D_Similarity_Linalg: URM Detected 63 (0.79 %) cold users.
CFW_D_Similarity_Linalg: URM Detected 2346 (9.03 %) cold items.
CFW_D_Similarity_Linalg: Generating train data
Unable to load Cython Compute_Similarity, reverting to Python


KeyboardInterrupt: ignored

### iALS

In [ ]:
!pip install implicit

  Using cached https://files.pythonhosted.org/packages/bc/07/c0121884722d16e2c5beeb815f6b84b41cbf22e738e4075f1475be2791bc/implicit-0.4.4.tar.gz
  Created wheel for implicit: filename=implicit-0.4.4-cp36-cp36m-linux_x86_64.whl size=3419438 sha256=e0995bebae2407e97c01f470ade33e3c44693188b51e442d622f499cabc68336
  Stored in directory: /root/.cache/pip/wheels/bf/d4/ec/fd4f622fcbefb7521f149905295b2c26adecb23af38aa28217
Successfully built implicit


In [ ]:
from MatrixFactorization.IALSRecommender_implicit import IALSRecommender

iALS = IALSRecommender(URM_and_ICM)
iALS.fit(**best_parameters_rec_9)
evaluator_validation.evaluateRecommender(iALS)

IALSRecommender: URM Detected 83 (0.30 %) cold users.



EvaluatorHoldout: Processed 5664 ( 100.00% ) in 5.69 sec. Users per second: 995


({5: {'ARHR': 0.16279131355932333,
   'AVERAGE_POPULARITY': 0.11889641036240878,
   'COVERAGE_ITEM': 0.3324735322425409,
   'COVERAGE_ITEM_CORRECT': 0.04515880654475457,
   'COVERAGE_USER': 0.7127217818044546,
   'COVERAGE_USER_CORRECT': 0.17390210142192022,
   'DIVERSITY_GINI': 0.14792318887578104,
   'DIVERSITY_HERFINDAHL': 0.9993970866649111,
   'DIVERSITY_MEAN_INTER_LIST': 0.9971614858467742,
   'F1': 0.07800595259360969,
   'HIT_RATE': 0.2978460451977401,
   'MAP': 0.07399761652542373,
   'MRR': 0.1462776600753303,
   'NDCG': 0.09454496374411646,
   'NOVELTY': 0.0026905986956674726,
   'PRECISION': 0.05956920903954666,
   'PRECISION_RECALL_MIN_DEN': 0.12464983521657272,
   'RECALL': 0.1129704441864207,
   'ROC_AUC': 0.15010004708097927,
   'SHANNON_ENTROPY': 12.010041781717188}},
 'CUTOFF: 5 - ROC_AUC: 0.1501000, PRECISION: 0.0595692, PRECISION_RECALL_MIN_DEN: 0.1246498, RECALL: 0.1129704, MAP: 0.0739976, MRR: 0.1462777, NDCG: 0.0945450, F1: 0.0780060, HIT_RATE: 0.2978460, ARHR: 0

### FunkSVD

In [ ]:
from MatrixFactorization.Cython.MatrixFactorization_Cython import _MatrixFactorization_Cython
funkSVD = _MatrixFactorization_Cython(urm_train, "FUNK_SVD")
funkSVD.fit(**best_parameters_rec_10)
evaluator_validation.evaluateRecommender(funkSVD)

MatrixFactorization_Cython_Recommender: URM Detected 63 (0.79 %) cold users.
MatrixFactorization_Cython_Recommender: URM Detected 2346 (9.03 %) cold items.
FUNK_SVD: Processed 90510 ( 99.88% ) in 0.70 seconds. MSE loss 9.10E-01. Sample per second: 129041
FUNK_SVD: Epoch 1 of 10. Elapsed time 0.45 sec
FUNK_SVD: Processed 90510 ( 99.88% ) in 1.15 seconds. MSE loss 7.33E-01. Sample per second: 78876
FUNK_SVD: Epoch 2 of 10. Elapsed time 0.89 sec
FUNK_SVD: Processed 90510 ( 99.88% ) in 0.68 seconds. MSE loss 5.78E-01. Sample per second: 133737
FUNK_SVD: Epoch 3 of 10. Elapsed time 1.42 sec
FUNK_SVD: Processed 90510 ( 99.88% ) in 1.27 seconds. MSE loss 4.47E-01. Sample per second: 71321
FUNK_SVD: Epoch 4 of 10. Elapsed time 2.02 sec
FUNK_SVD: Processed 90510 ( 99.88% ) in 0.74 seconds. MSE loss 3.38E-01. Sample per second: 121852
FUNK_SVD: Epoch 5 of 10. Elapsed time 2.49 sec
FUNK_SVD: Processed 90510 ( 99.88% ) in 1.21 seconds. MSE loss 2.48E-01. Sample per second: 75010
FUNK_SVD: Epoch 6 

({5: {'ROC_AUC': 0.019627114605670716,
   'PRECISION': 0.005718370264474611,
   'PRECISION_RECALL_MIN_DEN': 0.011252084822492276,
   'RECALL': 0.010084493266141527,
   'MAP': 0.007120165197363202,
   'MRR': 0.01801882296878723,
   'NDCG': 0.009358347455150127,
   'F1': 0.00729828063295941,
   'HIT_RATE': 0.028591851322373123,
   'ARHR': 0.018367286156778653,
   'NOVELTY': 0.0019576150504769896,
   'AVERAGE_POPULARITY': 0.4195045511712708,
   'DIVERSITY_MEAN_INTER_LIST': 0.6824894585114735,
   'DIVERSITY_HERFINDAHL': 0.9364734996558861,
   'COVERAGE_ITEM': 0.04519730510105871,
   'COVERAGE_ITEM_CORRECT': 0.000846968238691049,
   'COVERAGE_USER': 0.7041650937460677,
   'COVERAGE_USER_CORRECT': 0.01937838177928778,
   'DIVERSITY_GINI': 0.004722523325539625,
   'SHANNON_ENTROPY': 5.983399787993944}},
 'CUTOFF: 5 - ROC_AUC: 0.0196271, PRECISION: 0.0057184, PRECISION_RECALL_MIN_DEN: 0.0112521, RECALL: 0.0100845, MAP: 0.0071202, MRR: 0.0180188, NDCG: 0.0093583, F1: 0.0072983, HIT_RATE: 0.0285

### AsymmetricSVD

In [ ]:
from MatrixFactorization.Cython.MatrixFactorization_Cython import _MatrixFactorization_Cython
asymmetricSVD = _MatrixFactorization_Cython(urm_train, "ASY_SVD")
asymmetricSVD.fit(**best_parameters_rec_11)
evaluator_validation.evaluateRecommender(asymmetricSVD)

MatrixFactorization_Cython_Recommender: URM Detected 63 (0.79 %) cold users.
MatrixFactorization_Cython_Recommender: URM Detected 2346 (9.03 %) cold items.
ASY_SVD: Processed 90614 ( 100.00% ) in 2.33 seconds. MSE loss 4.94E+01. Sample per second: 38931
ASY_SVD: Epoch 1 of 1. Elapsed time 1.81 sec
ASY_SVD: Terminating at epoch 1. Elapsed time 1.82 sec
EvaluatorHoldout: Processed 5596 ( 100.00% ) in 7.34 sec. Users per second: 762


({5: {'ROC_AUC': 0.002546461758398856,
   'PRECISION': 0.000857755539671194,
   'PRECISION_RECALL_MIN_DEN': 0.001042411246128187,
   'RECALL': 0.0005624382706207416,
   'MAP': 0.0005283535858946868,
   'MRR': 0.0022962830593280916,
   'NDCG': 0.0005808366736193442,
   'F1': 0.0006793925432597933,
   'HIT_RATE': 0.004288777698355968,
   'ARHR': 0.0022962830593280916,
   'NOVELTY': 0.0023685889316574543,
   'AVERAGE_POPULARITY': 0.07852659223134058,
   'DIVERSITY_MEAN_INTER_LIST': 0.9615577065451448,
   'DIVERSITY_HERFINDAHL': 0.9922771754152997,
   'COVERAGE_ITEM': 0.027256977863330124,
   'COVERAGE_ITEM_CORRECT': 0.0006159769008662175,
   'COVERAGE_USER': 0.7041650937460677,
   'COVERAGE_USER_CORRECT': 0.003020007550018875,
   'DIVERSITY_GINI': 0.007341240707593286,
   'SHANNON_ENTROPY': 7.900781459335291}},
 'CUTOFF: 5 - ROC_AUC: 0.0025465, PRECISION: 0.0008578, PRECISION_RECALL_MIN_DEN: 0.0010424, RECALL: 0.0005624, MAP: 0.0005284, MRR: 0.0022963, NDCG: 0.0005808, F1: 0.0006794, HIT_

### MF BPR

In [ ]:
from MatrixFactorization.Cython.MatrixFactorization_Cython import _MatrixFactorization_Cython
MF_BPR = _MatrixFactorization_Cython(urm_train, "MF_BPR")
MF_BPR.fit(**best_parameters_rec_12)
evaluator_validation.evaluateRecommender(MF_BPR)

MatrixFactorization_Cython_Recommender: URM Detected 63 (0.79 %) cold users.
MatrixFactorization_Cython_Recommender: URM Detected 2346 (9.03 %) cold items.
MF_BPR: Processed 6144 ( 75.00% ) in 0.64 seconds. BPR loss 1.07E+00. Sample per second: 9645
MF_BPR: Epoch 1 of 164. Elapsed time 0.53 sec
MF_BPR: Processed 6144 ( 75.00% ) in 1.13 seconds. BPR loss 4.00E+00. Sample per second: 5420
MF_BPR: Epoch 2 of 164. Elapsed time 1.02 sec
MF_BPR: Processed 6144 ( 75.00% ) in 0.63 seconds. BPR loss 6.76E+00. Sample per second: 9686
MF_BPR: Epoch 3 of 164. Elapsed time 1.52 sec
MF_BPR: Processed 6144 ( 75.00% ) in 1.16 seconds. BPR loss 9.01E+00. Sample per second: 5287
MF_BPR: Epoch 4 of 164. Elapsed time 2.05 sec
MF_BPR: Processed 6144 ( 75.00% ) in 0.77 seconds. BPR loss 1.10E+01. Sample per second: 7962
MF_BPR: Epoch 5 of 164. Elapsed time 2.66 sec
MF_BPR: Processed 6144 ( 75.00% ) in 1.38 seconds. BPR loss 1.25E+01. Sample per second: 4450
MF_BPR: Epoch 6 of 164. Elapsed time 3.27 sec
MF_B

MF_BPR: Processed 6144 ( 75.00% ) in 0.64 seconds. BPR loss 4.09E+01. Sample per second: 9639
MF_BPR: Epoch 58 of 164. Elapsed time 29.53 sec
MF_BPR: Processed 6144 ( 75.00% ) in 1.22 seconds. BPR loss 4.16E+01. Sample per second: 5036
MF_BPR: Epoch 59 of 164. Elapsed time 30.11 sec
MF_BPR: Processed 6144 ( 75.00% ) in 0.74 seconds. BPR loss 4.19E+01. Sample per second: 8295
MF_BPR: Epoch 60 of 164. Elapsed time 30.63 sec
MF_BPR: Processed 6144 ( 75.00% ) in 1.23 seconds. BPR loss 4.21E+01. Sample per second: 4978
MF_BPR: Epoch 61 of 164. Elapsed time 31.12 sec
MF_BPR: Processed 6144 ( 75.00% ) in 0.74 seconds. BPR loss 4.19E+01. Sample per second: 8348
MF_BPR: Epoch 62 of 164. Elapsed time 31.62 sec
MF_BPR: Processed 6144 ( 75.00% ) in 1.23 seconds. BPR loss 4.25E+01. Sample per second: 4995
MF_BPR: Epoch 63 of 164. Elapsed time 32.12 sec
MF_BPR: Processed 6144 ( 75.00% ) in 0.73 seconds. BPR loss 4.27E+01. Sample per second: 8461
MF_BPR: Epoch 64 of 164. Elapsed time 32.61 sec
MF_BPR

MF_BPR: Epoch 115 of 164. Elapsed time 58.23 sec
MF_BPR: Processed 6144 ( 75.00% ) in 0.86 seconds. BPR loss 5.36E+01. Sample per second: 7129
MF_BPR: Epoch 116 of 164. Elapsed time 58.75 sec
MF_BPR: Processed 6144 ( 75.00% ) in 1.36 seconds. BPR loss 5.41E+01. Sample per second: 4518
MF_BPR: Epoch 117 of 164. Elapsed time 59.25 sec
MF_BPR: Processed 6144 ( 75.00% ) in 0.84 seconds. BPR loss 5.42E+01. Sample per second: 7277
MF_BPR: Epoch 118 of 164. Elapsed time 59.73 sec
MF_BPR: Processed 6144 ( 75.00% ) in 1.35 seconds. BPR loss 5.45E+01. Sample per second: 4565
MF_BPR: Epoch 119 of 164. Elapsed time 1.00 min
MF_BPR: Processed 6144 ( 75.00% ) in 0.85 seconds. BPR loss 5.51E+01. Sample per second: 7198
MF_BPR: Epoch 120 of 164. Elapsed time 1.01 min
MF_BPR: Processed 6144 ( 75.00% ) in 1.36 seconds. BPR loss 5.51E+01. Sample per second: 4522
MF_BPR: Epoch 121 of 164. Elapsed time 1.02 min
MF_BPR: Processed 6144 ( 75.00% ) in 0.86 seconds. BPR loss 5.52E+01. Sample per second: 7153
MF

({5: {'ROC_AUC': 0.03484631879914224,
   'PRECISION': 0.0130450321658328,
   'PRECISION_RECALL_MIN_DEN': 0.023335120324041028,
   'RECALL': 0.01982978654389413,
   'MAP': 0.013381284250655242,
   'MRR': 0.03283893257088395,
   'NDCG': 0.01752833728468827,
   'F1': 0.015737285464035706,
   'HIT_RATE': 0.06522516082916369,
   'ARHR': 0.03512330235882771,
   'NOVELTY': 0.002394274244509381,
   'AVERAGE_POPULARITY': 0.09884625998678975,
   'DIVERSITY_MEAN_INTER_LIST': 0.9947242924059763,
   'DIVERSITY_HERFINDAHL': 0.9989093072198512,
   'COVERAGE_ITEM': 0.26571703561116455,
   'COVERAGE_ITEM_CORRECT': 0.007199230028873917,
   'COVERAGE_USER': 0.7041650937460677,
   'COVERAGE_USER_CORRECT': 0.040392600981502456,
   'DIVERSITY_GINI': 0.09915735160382277,
   'SHANNON_ENTROPY': 11.352430044230072}},
 'CUTOFF: 5 - ROC_AUC: 0.0348463, PRECISION: 0.0130450, PRECISION_RECALL_MIN_DEN: 0.0233351, RECALL: 0.0198298, MAP: 0.0133813, MRR: 0.0328389, NDCG: 0.0175283, F1: 0.0157373, HIT_RATE: 0.0652252, 

### SLIM ElasticNet

In [ ]:
from SLIM_ElasticNet.SLIMElasticNetRecommender import SLIMElasticNetRecommender
SLIM_EN = SLIMElasticNetRecommender(urm_train)
SLIM_EN.fit(**best_parameters_rec_13)

SLIMElasticNetRecommender: URM Detected 63 (0.79 %) cold users.
SLIMElasticNetRecommender: URM Detected 2346 (9.03 %) cold items.
SLIMElasticNetRecommender: Processed 5238 ( 20.17% ) in 5.00 min. Items per second: 17.45
SLIMElasticNetRecommender: Processed 10615 ( 40.87% ) in 10.00 min. Items per second: 17.69
SLIMElasticNetRecommender: Processed 16072 ( 61.87% ) in 15.00 min. Items per second: 17.85
SLIMElasticNetRecommender: Processed 21606 ( 83.18% ) in 20.00 min. Items per second: 18.00
SLIMElasticNetRecommender: Processed 25975 ( 100.00% ) in 24.01 min. Items per second: 18.03


### ItemKNNCBF with stack

In [ ]:
from KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
itemKNNCBF_stack = ItemKNNCBFRecommender(urm_train, ICM_and_URM)
itemKNNCBF_stack.fit(**best_parameters_rec_14)
evaluator_validation.evaluateRecommender(itemKNNCBF_stack)

ItemKNNCBFRecommender: URM Detected 63 (0.79 %) cold users.
ItemKNNCBFRecommender: URM Detected 2346 (9.03 %) cold items.
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 25700 ( 99 % ), 854.94 column/sec, elapsed time 0.50 min
Similarity column 25975 ( 100 % ), 855.14 column/sec, elapsed time 0.51 min
EvaluatorHoldout: Processed 5596 ( 100.00% ) in 4.36 sec. Users per second: 1282


({5: {'ARHR': 0.149002263521564,
   'AVERAGE_POPULARITY': 0.10526212213063584,
   'COVERAGE_ITEM': 0.2965928777670837,
   'COVERAGE_ITEM_CORRECT': 0.03518768046198267,
   'COVERAGE_USER': 0.7041650937460677,
   'COVERAGE_USER_CORRECT': 0.15930539826349566,
   'DIVERSITY_GINI': 0.10816365818956872,
   'DIVERSITY_HERFINDAHL': 0.9982791726552357,
   'DIVERSITY_MEAN_INTER_LIST': 0.9915730564599635,
   'F1': 0.07052553397854153,
   'HIT_RATE': 0.27519656897784134,
   'MAP': 0.06532657255182257,
   'MRR': 0.1340243030736246,
   'NDCG': 0.08374052892390471,
   'NOVELTY': 0.0023946482946692704,
   'PRECISION': 0.055039313795567174,
   'PRECISION_RECALL_MIN_DEN': 0.10986418870621865,
   'RECALL': 0.09813839173401703,
   'ROC_AUC': 0.13896831069811774,
   'SHANNON_ENTROPY': 11.16483039866974}},
 'CUTOFF: 5 - ROC_AUC: 0.1389683, PRECISION: 0.0550393, PRECISION_RECALL_MIN_DEN: 0.1098642, RECALL: 0.0981384, MAP: 0.0653266, MRR: 0.1340243, NDCG: 0.0837405, F1: 0.0705255, HIT_RATE: 0.2751966, ARHR: 0

## Hybrid recommender

In [ ]:
!pip install scikit-optimize

     |████████████████████████████████| 102kB 6.2MB/s 


In [ ]:
from Base.BaseSimilarityMatrixRecommender import BaseItemSimilarityMatrixRecommender
from Base.Recommender_utils import check_matrix

class ItemKNNScoresHybridRecommender_2(BaseItemSimilarityMatrixRecommender):
    """ ItemKNNScoresHybridRecommender
    Hybrid of two prediction scores R = R1*alpha + R2*(1-alpha)

    """

    RECOMMENDER_NAME = "ItemKNNScoresHybridRecommender_2"


    def __init__(self, URM_train, Recommender_1, Recommender_2):
        super(ItemKNNScoresHybridRecommender_2, self).__init__(URM_train)

        self.URM_train = check_matrix(URM_train.copy(), 'csr')
        self.Recommender_1 = Recommender_1
        self.Recommender_2 = Recommender_2

        
    def fit(self, alpha = 0.5, beta = 0.5):

        self.alpha = alpha        
        self.beta = beta 

    def _compute_item_score(self, user_id_array, items_to_compute = None):
        
        item_weights_1 = self.Recommender_1._compute_item_score(user_id_array)
        item_weights_2 = self.Recommender_2._compute_item_score(user_id_array)

        item_weights = item_weights_1*self.alpha + item_weights_2*self.beta 

        return item_weights

In [ ]:
from Base.BaseSimilarityMatrixRecommender import BaseItemSimilarityMatrixRecommender
from Base.Recommender_utils import check_matrix

class ItemKNNScoresHybridRecommender_3(BaseItemSimilarityMatrixRecommender):
    """ ItemKNNScoresHybridRecommender
    Hybrid of two prediction scores R = R1*alpha + R2*(1-alpha)

    """

    RECOMMENDER_NAME = "ItemKNNScoresHybridRecommender_3"


    def __init__(self, URM_train, Recommender_1, Recommender_2, Recommender_3):
        super(ItemKNNScoresHybridRecommender_3, self).__init__(URM_train)

        self.URM_train = check_matrix(URM_train.copy(), 'csr')
        self.Recommender_1 = Recommender_1
        self.Recommender_2 = Recommender_2
        self.Recommender_3 = Recommender_3

        
    def fit(self, alpha = 0.5, beta = 0.5, gamma = 0.5):

        self.alpha = alpha      
        self.beta = beta      
        self.gamma = gamma        


    def _compute_item_score(self, user_id_array, items_to_compute = None):
        
        item_weights_1 = self.Recommender_1._compute_item_score(user_id_array)
        item_weights_2 = self.Recommender_2._compute_item_score(user_id_array)
        item_weights_3 = self.Recommender_3._compute_item_score(user_id_array)

        item_weights = item_weights_1*self.alpha + item_weights_2*self.beta + item_weights_3*self.gamma

        return item_weights

In [ ]:
from Base.BaseSimilarityMatrixRecommender import BaseItemSimilarityMatrixRecommender
from Base.Recommender_utils import check_matrix

class ItemKNNScoresHybridRecommender_4(BaseItemSimilarityMatrixRecommender):
    """ ItemKNNScoresHybridRecommender
    Hybrid of two prediction scores R = R1*alpha + R2*(1-alpha)

    """

    RECOMMENDER_NAME = "ItemKNNScoresHybridRecommender_4"


    def __init__(self, URM_train, Recommender_1, Recommender_2, Recommender_3, Recommender_4):
        super(ItemKNNScoresHybridRecommender_4, self).__init__(URM_train)

        self.URM_train = check_matrix(URM_train.copy(), 'csr')
        self.Recommender_1 = Recommender_1
        self.Recommender_2 = Recommender_2
        self.Recommender_3 = Recommender_3
        self.Recommender_4 = Recommender_4

        
    def fit(self, alpha = 0.5, beta = 0.5, gamma = 0.5, delta = 0.5):

        self.alpha = alpha      
        self.beta = beta      
        self.gamma = gamma      
        self.delta = delta


    def _compute_item_score(self, user_id_array, items_to_compute = None):
        
        item_weights_1 = self.Recommender_1._compute_item_score(user_id_array)
        item_weights_2 = self.Recommender_2._compute_item_score(user_id_array)
        item_weights_3 = self.Recommender_3._compute_item_score(user_id_array)
        item_weights_4 = self.Recommender_4._compute_item_score(user_id_array)

        item_weights = item_weights_1*self.alpha + item_weights_2*self.beta + item_weights_3*self.gamma + item_weights_4*self.delta

        return item_weights

In [ ]:
from Base.BaseSimilarityMatrixRecommender import BaseItemSimilarityMatrixRecommender
from Base.Recommender_utils import check_matrix

class ItemKNNScoresHybridRecommender_5(BaseItemSimilarityMatrixRecommender):
    """ ItemKNNScoresHybridRecommender
    Hybrid of two prediction scores R = R1*alpha + R2*(1-alpha)

    """

    RECOMMENDER_NAME = "ItemKNNScoresHybridRecommender_5"


    def __init__(self, URM_train, Recommender_1, Recommender_2, Recommender_3, Recommender_4, Recommender_5):
        super(ItemKNNScoresHybridRecommender_5, self).__init__(URM_train)

        self.URM_train = check_matrix(URM_train.copy(), 'csr')
        self.Recommender_1 = Recommender_1
        self.Recommender_2 = Recommender_2
        self.Recommender_3 = Recommender_3
        self.Recommender_4 = Recommender_4
        self.Recommender_5 = Recommender_5

        
    def fit(self, alpha = 0.5, beta = 0.5, gamma = 0.5, delta = 0.5, epsilon = 0.5):

        self.alpha = alpha      
        self.beta = beta      
        self.gamma = gamma      
        self.delta = delta
        self.epsilon = epsilon

    def _compute_item_score(self, user_id_array, items_to_compute = None):
        
        item_weights_1 = self.Recommender_1._compute_item_score(user_id_array)
        item_weights_2 = self.Recommender_2._compute_item_score(user_id_array)
        item_weights_3 = self.Recommender_3._compute_item_score(user_id_array)
        item_weights_4 = self.Recommender_4._compute_item_score(user_id_array)
        item_weights_5 = self.Recommender_5._compute_item_score(user_id_array)


        item_weights = item_weights_1*self.alpha + item_weights_2*self.beta + item_weights_3*self.gamma + item_weights_4*self.delta + item_weights_5*self.epsilon

        return item_weights

In [ ]:
from ParameterTuning.SearchBayesianSkopt import SearchBayesianSkopt

recommender_class = ItemKNNScoresHybridRecommender_3

parameterSearch = SearchBayesianSkopt(recommender_class,
                                      evaluator_validation=evaluator_validation)

In [ ]:
from ParameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
from skopt.space import Real, Integer, Categorical

hyperparameters_range_dictionary = {}
hyperparameters_range_dictionary["alpha"] = Real(low = 0, high = 1, prior = 'uniform')
hyperparameters_range_dictionary["beta"] = Real(low = 0, high = 1, prior = 'uniform')
hyperparameters_range_dictionary["gamma"] = Real(low = 0, high = 1, prior = 'uniform')
#hyperparameters_range_dictionary["delta"] = Real(low = 0, high = 1, prior = 'uniform')
#hyperparameters_range_dictionary["epsilon"] = Real(low = 0, high = 1, prior = 'uniform')

recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_and_ICM, userKNNCF, RP3beta, iALS],
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {}
)

output_folder_path = "result_experiments/"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

In [ ]:
n_cases = 70
metric_to_optimize = "MAP"

parameterSearch.search(recommender_input_args,
                       parameter_search_space = hyperparameters_range_dictionary,
                       n_cases = n_cases,
                       n_random_starts = 30,
                       save_model = "no",
                       output_folder_path = output_folder_path,
                       output_file_name_root = recommender_class.RECOMMENDER_NAME,
                       metric_to_optimize = metric_to_optimize
                      )

Iteration No: 1 started. Evaluating function at random point.
ItemKNNScoresHybridRecommender_3: URM Detected 83 (0.30 %) cold users.
SearchBayesianSkopt: Testing config: {'alpha': 0.33602252379991887, 'beta': 0.6777978047774833, 'gamma': 0.8269551719365362}
EvaluatorHoldout: Processed 5664 ( 100.00% ) in 8.56 sec. Users per second: 661
SearchBayesianSkopt: New best config found. Config 0: {'alpha': 0.33602252379991887, 'beta': 0.6777978047774833, 'gamma': 0.8269551719365362} - results: ROC_AUC: 0.1500706, PRECISION: 0.0581215, PRECISION_RECALL_MIN_DEN: 0.1189118, RECALL: 0.1068153, MAP: 0.0727542, MRR: 0.1436911, NDCG: 0.0919294, F1: 0.0752805, HIT_RATE: 0.2906073, ARHR: 0.1599488, NOVELTY: 0.0026484, AVERAGE_POPULARITY: 0.1491221, DIVERSITY_MEAN_INTER_LIST: 0.9922224, DIVERSITY_HERFINDAHL: 0.9984094, COVERAGE_ITEM: 0.3067950, COVERAGE_ITEM_CORRECT: 0.0378056, COVERAGE_USER: 0.7127218, COVERAGE_USER_CORRECT: 0.1692463, DIVERSITY_GINI: 0.1162839, SHANNON_ENTROPY: 11.2938793, 

Iteration

/usr/local/lib/python3.6/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


EvaluatorHoldout: Processed 5664 ( 100.00% ) in 8.49 sec. Users per second: 667
SearchBayesianSkopt: Config 38 is suboptimal. Config: {'alpha': 0.0, 'beta': 1.0, 'gamma': 1.0} - results: ROC_AUC: 0.1547640, PRECISION: 0.0621469, PRECISION_RECALL_MIN_DEN: 0.1292314, RECALL: 0.1168471, MAP: 0.0774051, MRR: 0.1519215, NDCG: 0.0984360, F1: 0.0811389, HIT_RATE: 0.3107345, ARHR: 0.1695327, NOVELTY: 0.0026945, AVERAGE_POPULARITY: 0.1215462, DIVERSITY_MEAN_INTER_LIST: 0.9965191, DIVERSITY_HERFINDAHL: 0.9992686, COVERAGE_ITEM: 0.3516073, COVERAGE_ITEM_CORRECT: 0.0448123, COVERAGE_USER: 0.7127218, COVERAGE_USER_CORRECT: 0.1794388, DIVERSITY_GINI: 0.1580581, SHANNON_ENTROPY: 12.0133554, 

Iteration No: 39 ended. Search finished for the next optimal point.
Time taken: 8.8859
Function value obtained: -0.0774
Current minimum: -0.0774
Iteration No: 40 started. Searching for the next optimal point.
ItemKNNScoresHybridRecommender_3: URM Detected 83 (0.30 %) cold users.
SearchBayesianSkopt: Testing conf

/usr/local/lib/python3.6/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


EvaluatorHoldout: Processed 5664 ( 100.00% ) in 8.55 sec. Users per second: 662
SearchBayesianSkopt: Config 44 is suboptimal. Config: {'alpha': 0.0, 'beta': 1.0, 'gamma': 1.0} - results: ROC_AUC: 0.1547640, PRECISION: 0.0621469, PRECISION_RECALL_MIN_DEN: 0.1292314, RECALL: 0.1168471, MAP: 0.0774051, MRR: 0.1519215, NDCG: 0.0984360, F1: 0.0811389, HIT_RATE: 0.3107345, ARHR: 0.1695327, NOVELTY: 0.0026945, AVERAGE_POPULARITY: 0.1215462, DIVERSITY_MEAN_INTER_LIST: 0.9965191, DIVERSITY_HERFINDAHL: 0.9992686, COVERAGE_ITEM: 0.3516073, COVERAGE_ITEM_CORRECT: 0.0448123, COVERAGE_USER: 0.7127218, COVERAGE_USER_CORRECT: 0.1794388, DIVERSITY_GINI: 0.1580581, SHANNON_ENTROPY: 12.0133554, 

Iteration No: 45 ended. Search finished for the next optimal point.
Time taken: 8.9999
Function value obtained: -0.0774
Current minimum: -0.0774
Iteration No: 46 started. Searching for the next optimal point.
ItemKNNScoresHybridRecommender_3: URM Detected 83 (0.30 %) cold users.
SearchBayesianSkopt: Testing conf

...

EvaluatorHoldout: Processed 5664 ( 100.00% ) in 8.47 sec. Users per second: 668
SearchBayesianSkopt: Config 65 is suboptimal. Config: {'alpha': 0.0, 'beta': 1.0, 'gamma': 1.0} - results: ROC_AUC: 0.1547640, PRECISION: 0.0621469, PRECISION_RECALL_MIN_DEN: 0.1292314, RECALL: 0.1168471, MAP: 0.0774051, MRR: 0.1519215, NDCG: 0.0984360, F1: 0.0811389, HIT_RATE: 0.3107345, ARHR: 0.1695327, NOVELTY: 0.0026945, AVERAGE_POPULARITY: 0.1215462, DIVERSITY_MEAN_INTER_LIST: 0.9965191, DIVERSITY_HERFINDAHL: 0.9992686, COVERAGE_ITEM: 0.3516073, COVERAGE_ITEM_CORRECT: 0.0448123, COVERAGE_USER: 0.7127218, COVERAGE_USER_CORRECT: 0.1794388, DIVERSITY_GINI: 0.1580581, SHANNON_ENTROPY: 12.0133554, 

Iteration No: 66 ended. Search finished for the next optimal point.
Time taken: 9.1486
Function value obtained: -0.0774
Current minimum: -0.0774
Iteration No: 67 started. Searching for the next optimal point.
ItemKNNScoresHybridRecommender_3: URM Detected 83 (0.30 %) cold users.
SearchBayesianSkopt: Testing conf

/usr/local/lib/python3.6/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


EvaluatorHoldout: Processed 5664 ( 100.00% ) in 8.63 sec. Users per second: 657
SearchBayesianSkopt: Config 66 is suboptimal. Config: {'alpha': 0.0, 'beta': 1.0, 'gamma': 1.0} - results: ROC_AUC: 0.1547640, PRECISION: 0.0621469, PRECISION_RECALL_MIN_DEN: 0.1292314, RECALL: 0.1168471, MAP: 0.0774051, MRR: 0.1519215, NDCG: 0.0984360, F1: 0.0811389, HIT_RATE: 0.3107345, ARHR: 0.1695327, NOVELTY: 0.0026945, AVERAGE_POPULARITY: 0.1215462, DIVERSITY_MEAN_INTER_LIST: 0.9965191, DIVERSITY_HERFINDAHL: 0.9992686, COVERAGE_ITEM: 0.3516073, COVERAGE_ITEM_CORRECT: 0.0448123, COVERAGE_USER: 0.7127218, COVERAGE_USER_CORRECT: 0.1794388, DIVERSITY_GINI: 0.1580581, SHANNON_ENTROPY: 12.0133554, 

Iteration No: 67 ended. Search finished for the next optimal point.
Time taken: 9.2774
Function value obtained: -0.0774
Current minimum: -0.0774
Iteration No: 68 started. Searching for the next optimal point.
ItemKNNScoresHybridRecommender_3: URM Detected 83 (0.30 %) cold users.
SearchBayesianSkopt: Testing conf

/usr/local/lib/python3.6/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


EvaluatorHoldout: Processed 5664 ( 100.00% ) in 8.47 sec. Users per second: 668
SearchBayesianSkopt: Config 69 is suboptimal. Config: {'alpha': 0.0, 'beta': 1.0, 'gamma': 1.0} - results: ROC_AUC: 0.1547640, PRECISION: 0.0621469, PRECISION_RECALL_MIN_DEN: 0.1292314, RECALL: 0.1168471, MAP: 0.0774051, MRR: 0.1519215, NDCG: 0.0984360, F1: 0.0811389, HIT_RATE: 0.3107345, ARHR: 0.1695327, NOVELTY: 0.0026945, AVERAGE_POPULARITY: 0.1215462, DIVERSITY_MEAN_INTER_LIST: 0.9965191, DIVERSITY_HERFINDAHL: 0.9992686, COVERAGE_ITEM: 0.3516073, COVERAGE_ITEM_CORRECT: 0.0448123, COVERAGE_USER: 0.7127218, COVERAGE_USER_CORRECT: 0.1794388, DIVERSITY_GINI: 0.1580581, SHANNON_ENTROPY: 12.0133554, 

Iteration No: 70 ended. Search finished for the next optimal point.
Time taken: 9.1441
Function value obtained: -0.0774
Current minimum: -0.0774
SearchBayesianSkopt: Search complete. Best config is 33: {'alpha': 0.0, 'beta': 1.0, 'gamma': 1.0}



In [ ]:
from Base.DataIO import DataIO

data_loader = DataIO(folder_path = output_folder_path)
search_metadata = data_loader.load_data(recommender_class.RECOMMENDER_NAME + "_metadata.zip")

In [ ]:
best_parameters_hybrid = search_metadata["hyperparameters_best"]
best_parameters_hybrid

{'alpha': 0.0, 'beta': 1.0, 'gamma': 1.0}

In [ ]:
hybrid = ItemKNNScoresHybridRecommender_3(URM_and_ICM, userKNNCF, RP3beta, iALS)
hybrid.fit(**best_parameters_hybrid)

ItemKNNScoresHybridRecommender_3: URM Detected 83 (0.30 %) cold users.


In [ ]:
MAP = evaluator_validation.evaluateRecommender(hybrid)[0][5]["MAP"]
MAP

EvaluatorHoldout: Processed 5664 ( 100.00% ) in 8.55 sec. Users per second: 663


0.07740510240112976

## Recommend on the whole dataset

In [ ]:
urm_all = urm_train + urm_validation

In [ ]:
urm_all_stack_icm_all = sp.vstack([urm_all, icm_all.T], format="csr")

In [ ]:
itemKNNCF = ItemKNNCFRecommender(urm_all_stack_icm_all)
itemKNNCF.fit(**best_parameters_rec_1)

NameError: ignored

In [ ]:
userKNNCF = UserKNNCFRecommender(urm_all_stack_icm_all)
userKNNCF.fit(**best_parameters_rec_2)

UserKNNCFRecommender: URM Detected 2 (0.01 %) cold users.
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 27947 ( 100 % ), 1231.93 column/sec, elapsed time 0.38 min


In [ ]:
itemKNNCBF = ItemKNNCBFRecommender(urm_all, icm_all)
itemKNNCBF.fit(**best_parameters_rec_3)

In [ ]:
pureSVD = PureSVDItemRecommender(urm_all)
pureSVD.fit(**best_parameters_rec_4)

In [ ]:
P3alpha = P3alphaRecommender(urm_all)
P3alpha.fit(**best_parameters_rec_5)

P3alphaRecommender: URM Detected 1079 (4.15 %) cold items.


In [ ]:
slim_BPR = SLIM_BPR_Cython(urm_all)
slim_BPR.fit(**best_parameters_rec_6)

In [ ]:
RP3beta = RP3betaRecommender(urm_all_stack_icm_all)
RP3beta.fit(**best_parameters_rec_7)

RP3betaRecommender: URM Detected 2 (0.01 %) cold users.
RP3betaRecommender: Processed 24400 ( 93.94% ) in 1.01 minutes. Rows per second: 404


In [ ]:
CFW = CFW_D_Similarity_Linalg(urm_all, icm_all, itemKNNCF.W_sparse)
CFW.fit(**best_parameters_rec_8)

In [ ]:
SLIM_EN = SLIMElasticNetRecommender(urm_all)
SLIM_EN.fit(**best_parameters_rec_13)

In [ ]:
iALS = IALSRecommender(urm_all_stack_icm_all)
iALS.fit(**best_parameters_rec_9)

IALSRecommender: URM Detected 2 (0.01 %) cold users.


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-36-429827362abb>", line 2, in <module>
    iALS.fit(**best_parameters_rec_9)
  File "/content/drive/MyDrive/Competition_Recommender_Systems/MatrixFactorization/IALSRecommender_implicit.py", line 49, in fit
    model.fit(self.C)
  File "/usr/local/lib/python3.6/dist-packages/implicit/als.py", line 173, in fit
    num_threads=self.num_threads)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 1823, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most re

KeyboardInterrupt: ignored

In [ ]:
itemKNNCBF_stack = ItemKNNCBFRecommender(urm_all, sp.hstack([icm_all, urm_all.T], format="csr"))
itemKNNCBF_stack.fit(**best_parameters_rec_14)

In [ ]:
hybridrecommender = ItemKNNScoresHybridRecommender_3(urm_all_stack_icm_all, userKNNCF, RP3beta, iALS)
hybridrecommender.fit(**best_parameters_hybrid)

In [ ]:
import csv

def write_submission():
    submission_dir = os.path.join(cwd, 'submissions')
    csv_name = 'submission_'
    notebook_name = 'Hybrid_Recommender_userKNNCF_RP3beta_iALS_implicit'
    
    if not os.path.exists(submission_dir):
        os.makedirs(submission_dir)
        
    csv_name += str(len(os.listdir(submission_dir)) + 1)
    
    submission_dir = os.path.join(submission_dir, (csv_name + '_' + notebook_name))
    
    if not os.path.exists(submission_dir):
        os.makedirs(submission_dir)
    
    with open(os.path.join(submission_dir, (csv_name + '.txt')), 'w') as f:
        f.write(str(MAP))
        
    csv_name += '.csv'

    with open(os.path.join(submission_dir, csv_name), 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['user_id', 'item_list'])
        for userID in target_users:
            writer.writerow([userID, str(np.array(hybridrecommender.recommend(userID, 10)))[1:-1]])
    

In [ ]:
write_submission()